In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st 
import math
import statsmodels.api as sm
import pandas as pd
from scipy.optimize import root_scalar

In [10]:
def Brownian_Generator(W_0,m,T):
    """This function generartes a standard one dimensional brownian motion with time steps of size T/m

    Parameters
    ----------
    W_0 : float
        Initial value of the brownian motion
    m : int
        time steps of the brownian motion
    T : float
        time horizon of the brownian motion

    Returns
    -------
    W : np.ndarray
        BM evaluated at time (T/m,2T/m,...,T) (m_features,) """

    if 1 > m:
        return "Error: m must be bigger than 1"
    if type(m) != int:
        print("Error: m must be of type int")
    else:
        W = np.zeros(m)
        W[0] = W_0 + st.norm.rvs(loc = 0,scale = np.sqrt(T/m))
        for i in range(1,m):
            W[i] = W[i-1] + st.norm.rvs(loc = 0,scale = np.sqrt(T/m))
        return W

def GeometricBM_generator(W_0,m,S_0,sigma,r,T):
    """This function generates a geometric brownian motion with time steps of size T/m

    Parameters
    ----------
     W_0 : float
        Initial value of the brownian motion
    m : int
        time steps of the brownian motion
    S_0 : float
        Initial value of the geometric brownian motion
    sigma : float
        volatility of the geometric brownian motion
    r : float
        interest rate of the geometric brownian motion
    T : float
        time horizon of the geometric brownian motion

    Returns
    -------
    S : np.ndarray
        GBM evaluated at time (T/m,2T/m,...,T) (m_features,) """

    W = Brownian_Generator(W_0,m,T)
    S = np.zeros(m)

    for i in range(0,m):
        S[i] = S_0 * np.exp((r - 0.5*sigma**2) * (i*T/m) + sigma*W[i])
    return S

def f_european(S,K,r,T):
    """This function computes the payoff of an european put option at time T
    Parameters
    ----------
    S : np.ndarray
        GBM evaluated at time (T/m,2T/m,...,T) (m_features,)
    K : float
        strike price of the put option
    r : float
        interest rate of the put option
    T : float
        time horizon of the put option

    Returns
    -------
    f : float
        payoff of the put option at time T"""

    f = np.exp(-r*T) * np.amax( [0,(K - S[-1])] )
    return f

def f_asian(S,K,r,T):
    """ Path dependant discounted payoff of an asian put option at time T
    Parameters
    ----------
    S : np.ndarray
        GBM evaluated at time (T/m,2T/m,...,T) (m_features,)
    K : float
        strike price of the put option
    r : float
        interest rate of the put option
    T : float
        time horizon of the put option

    Returns
    -------
    f : float
        payoff of the put option at time T"""

    f = np.exp(-r*T) * np.amax( [0,(K - np.mean(S))] )
    return f

def J_estimator(sigma,N, K = 120, r = 0.05, T = 0.2, S_0 = 100, m = 1, W_0 = 0, I_market = 22):
    """ Crude MonteCarlo estimator for Robin_Monro algorithm
    Parameters
    ----------
    sigma : float
        volatility of the geometric brownian motion
    N : int
        Size of the sample size for the estimation of the expected value
    K : float
        strike price of the put option
    r : float
        interest rate of the put option
    T : float
        time horizon of the put option
    S_0 : float
        Initial value of the geometric brownian motion
    m : int
        time steps of the brownian motion
    W_0 : float
        Initial value of the brownian motion  
    I_market : float
        consisten mark to market price of the put option
    
    Returns
    -------
    J : float
        Crude MonteCarlo estimator of the the expected value of the payoff of the put option"""

    J = 0
    for i in range(N):
        S = GeometricBM_generator(W_0,m,S_0,sigma,r,T)
        J += ( np.exp(-r*T) * np.amax( [0,(K - S[-1])] ))
    J /= N
    J -= I_market
    return J

def I(sigma, I_market = 22 , S_0 = 100 , K = 120, r = 0.05, T = 0.2):
    """Closed form solution of an european put option of a geometric brownian motion with equidistant time steps adjusted to the market price
    Parameters
    ----------
    sigma : float
        volatility of the geometric brownian motion
    I_market : float
        Market price of such put options
    S_0 : float
        Initial value of the geometric brownian motion
    K : float
        strike price of the put option
    r : float
        interest rate of the put option
    sigma : float
        volatility of the geometric brownian motion
    T : float
        time horizon of the put option

    Returns
    -------
    I : float
        Mark to market price of the put option"""
    
    w = (np.log(K/S_0) - (r-0.5*sigma**2)*T) / ( sigma * np.sqrt(T) )
    I = np.exp(-r*T) * K * st.norm.cdf(w) - S_0 * st.norm.cdf(w - sigma*np.sqrt(T))
    return float(I-I_market)


def RM(func, n, N, rho ,sigma_0, alpha_0, sigma_optimal,z,M):
    """ Implementation of the Robbins-Monro algorithn for the root finding problem

    Parameters
    ----------
    func : function
        function for which we want to find the root 
    n : int
        number of iterations of the alogrithn
    N : int
        Size of the sample size for the estimation of the expected value 
    rho : float
        order convergence of the sequence alpha_n
    sigma_0 : float
        initial value of sigma estimated
    alpha_0 : float
        initial value of alpha sequence
    z : np.ndarray
        sequence of time steps at which we want to estimate the volatility
    M : int
        Stabiliztion of the MSE by averaging over M iterations
    Returns
    -------
    sigma_estim : float
        Estimation of the volatility of the geometric brownian motion at iteration n
    MSE : np.ndarray
        MSE of the estimated of the volatility at time given by index of z"""

    if np.amax(z) > n:
        return "Error: The time steps of evaluation must be smaller than n"
    if np.amin(z) < 2:
        return "Error: The time steps of evaluation must be bigger than 1"
    
    MSE = np.zeros(shape=(len(z),))

    for j in range(M):
        sigma_estim = sigma_0 - alpha_0 * func(sigma_0, N)

        for i in range(1,n):
            sigma_estim -= (alpha_0 / (i ** rho)) * func(sigma_estim, N) 

            if i in z:
                k = z.index(i)
                MSE[k] += ((sigma_estim - sigma_optimal) ** 2) 
    MSE /= M
        
    return sigma_estim, MSE

In [11]:
T = 0.2 #Maturity time
r = 0.05 #Interest rate
S_0 = 100 #Initial asset
K = 120 #Strike price
alpha_0 = 2 / (K + S_0) # Recommmended learning rate
I_market = 22 #Consistent mark-to-market price

In [12]:
W_0 = 0
m = 50
sigma = 4
W = Brownian_Generator(W_0,m,T)
S = GeometricBM_generator(W,S_0,sigma,r,T)

z = np.full(m,K, dtype=int)

plt.plot(S)
plt.plot(z, color = 'red', label = 'Strike price')
plt.legend()
plt.show()

print(f"At this computation, the payoff is {f_asian(S,K,r,T)} the mean of the payoff is {I(sigma,0,S_0,K,r,T)}")

TypeError: GeometricBM_generator() missing 1 required positional argument: 'T'

In [ ]:
E = [np.exp(r * i * T / m) for i in range(1, m + 1)]

# Calculate the set of possible values for I
I_values = [np.exp(-r * T) * (np.amax([K - S_0 / m * np.sum(E), 0])), np.exp(-r * T) * K]

print(f"The set of possible values of I is: {I_values}")

The set of possible values of I is: [19.294367043657672, 118.80598004990017]


Here there is no closed form solution so we cannot apply previous algorithm: so we will have to do a MC simulation of I to find the optimal sigma  

Applying RM algorithm:

In [ ]:
sigma_0 = 1 # Initial guess of sigma
rho = [0.8,1]
N = [1,10,100]
n = 1_000 # Number of iterations of the algorithm

z = [i * 20 for i in range(1,int(n/20))]
x = [i * 20 * N[0] for i in range(1,int(n/20))]

In [ ]:
sigma_0 = 1 # Initial guess of sigma
rho = [0.8,1]
N = [1,10,100]
n = 1_000 # Number of iterations of the algorithm
M = 100 # Number of iterations to average the MSE
z = [i * 20 for i in range(1,int(n/20))]
x = [i * 20 * N[0] for i in range(1,int(n/20))]


In [ ]:
for i in range(len(rho)):
    for j in range(len(N)):

        print(f"pho = {rho[i]} and N = {N[j]}")
        print(f"sigma_estimated = {round(sigma_estim[i,j],3)} and sigma_optimal = {round(result.root,3)}")
        print()
